# Análisis Interactivo de Modelos de Imputación

Este notebook proporciona una interfaz para visualizar y analizar los modelos de imputación generados por la pipeline de procesamiento de datos.

## Características principales:

- **Visualización interactiva** con Plotly Dash
- **Filtros encadenados** por aeronave, parámetro, tipo de modelo y predictores
- **Comparación visual** de modelos con puntos de entrenamiento y curvas de regresión
- **Panel informativo** con métricas detalladas y ecuaciones
- **Análisis robusto** de la calidad de los modelos de imputación

## Arquitectura del módulo:

```
Modulos/Analisis_modelos/
├── __init__.py                    # Punto de entrada del módulo
├── main_visualizacion_modelos.py  # Aplicación principal Dash
├── data_loader.py                 # Carga y procesamiento de datos
├── plot_utils.py                  # Utilidades de visualización
├── ui_components.py               # Componentes de interfaz
└── requirements.txt               # Dependencias del módulo
```

## 1. Estructura de Carpetas y Scripts

El módulo de análisis está organizado de forma modular para facilitar el mantenimiento y extensión:

### Scripts principales:

- **`main_visualizacion_modelos.py`**: Entrada principal que ejecuta la aplicación Dash
- **`data_loader.py`**: Funciones de carga y filtrado de datos del JSON
- **`plot_utils.py`**: Generación de gráficos interactivos con Plotly
- **`ui_components.py`**: Componentes visuales reutilizables para la interfaz

### Datos de entrada:
- **Archivo JSON**: `ADRpy/analisis/Results/modelos_completos_por_celda.json`
- **Estructura**: Contiene `modelos_por_celda` y `detalles_por_celda`

### Funcionalidades implementadas:
✅ Filtros encadenados (Aeronave → Parámetro → Tipo → Predictores)  
✅ Visualización de curvas de regresión sobre datos originales  
✅ Hover con métricas detalladas (MAPE, R², Confianza, etc.)  
✅ Panel lateral con información ampliada  
✅ Comparación entre tipos de modelos  
✅ Manejo de errores y advertencias

## 2. Carga y Procesamiento de Datos

El módulo `data_loader.py` maneja toda la lógica de carga y transformación de datos:

### Funciones principales:

- **`load_models_data()`**: Carga el archivo JSON y extrae los diccionarios principales
- **`extract_unique_values()`**: Obtiene valores únicos para filtros (aeronaves, parámetros, tipos)
- **`filter_models()`**: Filtra modelos según criterios especificados
- **`prepare_plot_data()`**: Prepara datos para visualización
- **`get_model_predictions()`**: Genera predicciones de modelos para curvas

### Estructura de datos procesados:

1. **modelos_por_celda**: Diccionario con clave `"Aeronave|Parámetro"`
   - Lista de modelos candidatos por celda
   - Cada modelo contiene: ecuaciones, métricas, datos de entrenamiento, advertencias

2. **detalles_por_celda**: Información consolidada de imputación final
   - Método seleccionado y valor imputado
   - Comparación entre métodos de similitud y correlación

In [1]:
# Importar módulos necesarios
import sys
import os
import json

# Añadir el directorio de módulos al path
modulos_path = os.path.join(os.getcwd(), 'Modulos')
if modulos_path not in sys.path:
    sys.path.append(modulos_path)

# Probar la carga de datos (sin ejecutar la app completa)
try:
    from Analisis_modelos.data_loader import load_models_data, extract_unique_values
    
    # Ruta al archivo JSON
    json_path = os.path.join('Results', 'modelos_completos_por_celda.json')
    
    if os.path.exists(json_path):
        print("✅ Archivo JSON encontrado")
        
        # Cargar datos
        modelos_por_celda, detalles_por_celda = load_models_data(json_path)
        
        print(f"📊 Modelos por celda: {len(modelos_por_celda)} celdas")
        print(f"📋 Detalles por celda: {len(detalles_por_celda)} celdas")
        
        # Extraer valores únicos para filtros
        unique_values = extract_unique_values(modelos_por_celda)
        
        print("\n🔍 Valores únicos disponibles:")
        for key, values in unique_values.items():
            print(f"  {key}: {len(values)} opciones")
            if len(values) <= 10:  # Mostrar si hay pocas opciones
                print(f"    {values}")
            else:
                print(f"    {values[:5]}... (y {len(values)-5} más)")
        
    else:
        print(f"❌ No se encontró el archivo JSON en: {json_path}")
        print("Verifique que la ruta sea correcta.")
        
except ImportError as e:
    print(f"⚠️ Error de importación: {e}")
    print("Instale las dependencias necesarias: pip install dash plotly pandas")
except Exception as e:
    print(f"❌ Error: {e}")

INFO:Analisis_modelos.data_loader:Cargados 6 celdas con modelos
INFO:Analisis_modelos.data_loader:Cargados 6 celdas con detalles
INFO:Analisis_modelos.data_loader:Cargados 6 celdas con detalles


✅ Archivo JSON encontrado
📊 Modelos por celda: 6 celdas
📋 Detalles por celda: 6 celdas

🔍 Valores únicos disponibles:
  aeronaves: 3 opciones
    ['A3', 'A5', 'A7']
  parametros: 5 opciones
    ['Alcance de la aeronave', 'Potencia HP', 'Velocidad a la que se realiza el crucero (KTAS)', 'envergadura', 'payload']
  tipos_modelo: 7 opciones
    ['exp-1', 'linear-1', 'linear-2', 'log-1', 'poly-1', 'poly-2', 'pot-1']
  predictores: 8 opciones
    ['Alcance de la aeronave', 'Ancho del fuselaje', 'Cantidad de motores', 'Potencia HP', 'Rango de comunicación', 'Velocidad a la que se realiza el crucero (KTAS)', 'envergadura', 'payload']
  n_predictores: 2 opciones
    [1, 2]


## 3. Definición de Filtros Encadenados

La aplicación implementa un sistema de filtros progresivos para explorar los datos:

### Jerarquía de filtros:

1. **Aeronave** → Muestra solo aeronaves con modelos disponibles
2. **Parámetro** → Filtra parámetros con imputaciones para la aeronave seleccionada  
3. **Tipo de modelo** → Permite seleccionar tipos específicos (linear, poly, log, exp, pot)
4. **Número de predictores** → Filtra por cantidad de variables predictoras
5. **Predictores específicos** → Selección de variables predictoras concretas

### Características de los filtros:

- **Encadenamiento inteligente**: Los filtros se actualizan dinámicamente
- **Selección múltiple**: Permite comparar varios tipos o modelos simultáneamente
- **Persistencia**: Las selecciones se mantienen al cambiar otros filtros
- **Validación**: Solo muestra opciones válidas según la selección actual

### Componentes de interfaz:

- **Dropdowns**: Para selecciones únicas (aeronave, parámetro)
- **Checklists**: Para selecciones múltiples (tipos, predictores)
- **Radio buttons**: Para opciones mutuamente excluyentes (tipo de comparación)

In [2]:
# Ejemplo de uso de filtros (sin ejecutar la aplicación completa)
try:
    from Analisis_modelos.data_loader import filter_models, get_parametros_for_aeronave
    
    if 'modelos_por_celda' in locals():
        # Ejemplo 1: Filtrar por aeronave específica
        if unique_values['aeronaves']:
            aeronave_ejemplo = unique_values['aeronaves'][0]
            print(f"🔍 Explorando aeronave: {aeronave_ejemplo}")
            
            # Obtener parámetros para esta aeronave
            parametros_aeronave = get_parametros_for_aeronave(modelos_por_celda, aeronave_ejemplo)
            print(f"   Parámetros disponibles: {parametros_aeronave}")
            
            if parametros_aeronave:
                parametro_ejemplo = parametros_aeronave[0]
                print(f"   Analizando parámetro: {parametro_ejemplo}")
                
                # Filtrar modelos para esta combinación
                modelos_filtrados = filter_models(
                    modelos_por_celda,
                    aeronave=aeronave_ejemplo,
                    parametro=parametro_ejemplo
                )
                
                celda_key = f"{aeronave_ejemplo}|{parametro_ejemplo}"
                if celda_key in modelos_filtrados:
                    modelos = modelos_filtrados[celda_key]
                    print(f"   📊 {len(modelos)} modelos encontrados")
                    
                    # Mostrar resumen de los primeros modelos
                    for i, modelo in enumerate(modelos[:3]):
                        if isinstance(modelo, dict):
                            tipo = modelo.get('tipo', 'N/A')
                            predictores = modelo.get('predictores', [])
                            mape = modelo.get('mape', 0)
                            r2 = modelo.get('r2', 0)
                            
                            print(f"     Modelo {i+1}: {tipo}")
                            print(f"       - Predictores: {', '.join(predictores)}")
                            print(f"       - MAPE: {mape:.3f}%, R²: {r2:.3f}")
        
        # Ejemplo 2: Filtrar por tipo de modelo
        print(f"\n🔍 Filtros por tipo de modelo:")
        tipos_disponibles = unique_values['tipos_modelo'][:3]  # Primeros 3 tipos
        
        modelos_por_tipo = filter_models(
            modelos_por_celda,
            tipos_modelo=tipos_disponibles
        )
        
        print(f"   Modelos filtrados por tipos {tipos_disponibles}: {len(modelos_por_tipo)} celdas")
        
    else:
        print("⚠️ No se han cargado los datos. Ejecute la celda anterior primero.")
        
except ImportError:
    print("⚠️ Módulo no disponible. Instale las dependencias necesarias.")
except Exception as e:
    print(f"❌ Error en filtros: {e}")

🔍 Explorando aeronave: A3
   Parámetros disponibles: ['Alcance de la aeronave', 'Potencia HP', 'Velocidad a la que se realiza el crucero (KTAS)', 'envergadura']
   Analizando parámetro: Alcance de la aeronave
   📊 21 modelos encontrados
     Modelo 1: linear-1
       - Predictores: payload
       - MAPE: 2.803%, R²: 0.934
     Modelo 2: log-1
       - Predictores: payload
       - MAPE: 2.980%, R²: 0.932
     Modelo 3: pot-1
       - Predictores: payload
       - MAPE: 2.602%, R²: 0.939

🔍 Filtros por tipo de modelo:
   Modelos filtrados por tipos ['exp-1', 'linear-1', 'linear-2']: 6 celdas


## 4. Generación de Gráficas Interactivas

El módulo `plot_utils.py` crea visualizaciones interactivas avanzadas:

### Características de las gráficas:

**Elementos visuales:**
- 📊 **Puntos originales**: Todos los datos del dataset (gris claro)
- 🔴 **Puntos de entrenamiento**: Datos usados para entrenar cada modelo (rojo, resaltados)
- 📈 **Curvas de regresión**: Líneas de predicción de cada modelo filtrado
- 🎨 **Colores diferenciados**: Cada tipo de modelo tiene su color y símbolo

**Información en hover:**
- ✅ **Ecuación del modelo** (texto plano y LaTeX)
- ✅ **Métricas de rendimiento** (MAPE, R², Correlación, Confianza)
- ✅ **Predictores utilizados** y número de muestras de entrenamiento
- ✅ **Advertencias** y validaciones del modelo
- ✅ **Indicación** si fue el modelo elegido para imputar

**Tipos de gráficos disponibles:**
1. **Vista principal**: Comparación visual de modelos con datos
2. **Vista de comparación**: Métricas lado a lado
3. **Vista de métricas**: Tablas detalladas de rendimiento

### Funciones implementadas:

- `create_interactive_plot()`: Gráfico principal con todos los elementos
- `add_model_curves()`: Añade curvas de regresión calculadas
- `create_comparison_plot()`: Gráficos de comparación entre modelos
- `create_metrics_summary_table()`: Tablas de resumen de métricas

## 5. Panel Lateral de Información Detallada

El módulo `ui_components.py` implementa un panel lateral rico en información:

### Secciones del panel:

**📋 Información Básica del Modelo:**
- Tipo de modelo y transformación aplicada
- Lista de predictores utilizados
- Número de predictores y muestras de entrenamiento

**📐 Ecuaciones Detalladas:**
- Ecuación en texto plano (unidades originales)
- Ecuación en formato LaTeX (para visualización matemática)
- Ecuación normalizada (para debugging)

**📊 Métricas de Rendimiento:**
- **MAPE**: Error absoluto porcentual medio
- **R²**: Coeficiente de determinación
- **Correlación**: Coeficiente de correlación combinado
- **Confianza**: Medida ajustada por penalización de complejidad

**⚠️ Advertencias y Validaciones:**
- Robustez del modelo
- Problemas de extrapolación
- Filtros de familia aplicados
- Warnings específicos del entrenamiento

**🎯 Estado de Imputación:**
- Indica si el modelo fue seleccionado para la imputación final
- Valor imputado y método empleado
- Comparación con otros métodos disponibles

### Características del panel:

- **Actualización dinámica**: Se actualiza automáticamente al seleccionar modelos
- **Formato legible**: Uso de componentes HTML estilizados
- **Información contextual**: Explica el significado de cada métrica
- **Navegación intuitiva**: Organizado en secciones colapsables

## 6. Gestión de Errores y Advertencias

El módulo implementa un sistema robusto de manejo de errores:

### Tipos de errores manejados:

**🔧 Errores de configuración:**
- Dependencias faltantes (Dash, Plotly, pandas)
- Archivo JSON no encontrado o corrupto
- Estructura de datos inconsistente

**📊 Errores de datos:**
- Claves faltantes en el JSON (`KeyError`)
- Valores nulos o indefinidos (`None`, `NaN`)
- Tipos de datos incompatibles
- Modelos sin datos de entrenamiento

**📈 Errores de visualización:**
- Predicciones con valores infinitos o NaN
- Rangos de datos vacíos o inválidos
- Ecuaciones no evaluables
- Problemas de renderizado de gráficos

### Estrategias de recuperación:

**🔄 Fallbacks inteligentes:**
- Uso de matplotlib si Dash no está disponible
- Interfaz por consola como alternativa
- Valores por defecto para campos faltantes
- Filtros automáticos de datos inválidos

**📝 Logging detallado:**
- Registro de errores con contexto
- Advertencias informativas para el usuario
- Mensajes de debug para desarrollo
- Métricas de uso y rendimiento

**🛡️ Validaciones preventivas:**
- Verificación de estructura JSON antes de procesar
- Validación de tipos de modelo soportados
- Comprobación de consistencia de datos
- Límites en rangos de predicción para evitar overflow

### Mensajes al usuario:

- **Informativos**: Progreso de carga y procesamiento
- **Advertencias**: Problemas menores que no impiden la ejecución  
- **Errores**: Problemas críticos con instrucciones de solución

## 7. Notebook de Ejemplo: Ejecución del Módulo

Esta sección muestra cómo ejecutar la aplicación interactiva completa.

### Opciones de ejecución:

**🚀 Aplicación Dash completa** (recomendado):
- Interfaz web interactiva en `http://localhost:8050`
- Todos los filtros y visualizaciones disponibles
- Panel lateral con información detallada
- Exportación de resultados

**💻 Versión simplificada** (alternativa):
- Interfaz por consola si Dash no está disponible
- Exploración básica de datos y métricas
- Útil para debugging y desarrollo

### Parámetros de configuración:

- **`json_path`**: Ruta al archivo JSON (por defecto: automática)
- **`use_dash`**: Usar interfaz Dash (True) o consola (False)  
- **`port`**: Puerto para la aplicación web (por defecto: 8050)
- **`debug`**: Modo debug con recarga automática (False para producción)

### Flujo de uso recomendado:

1. **Verificar dependencias** → Instalar si es necesario
2. **Probar carga de datos** → Validar archivo JSON
3. **Ejecutar aplicación** → Lanzar interfaz interactiva
4. **Explorar modelos** → Usar filtros para análisis específicos
5. **Comparar resultados** → Evaluar calidad de imputaciones

In [3]:
# Verificar e instalar dependencias necesarias
import subprocess
import sys

def install_package(package):
    """Instala un paquete usando pip."""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        return True
    except subprocess.CalledProcessError:
        return False

def check_dependencies():
    """Verifica si las dependencias están instaladas."""
    required_packages = {
        'dash': 'dash>=2.14.0',
        'plotly': 'plotly>=5.17.0', 
        'pandas': 'pandas>=1.5.0',
        'numpy': 'numpy>=1.21.0'
    }
    
    missing_packages = []
    
    for package, version_spec in required_packages.items():
        try:
            __import__(package)
            print(f"✅ {package} está instalado")
        except ImportError:
            print(f"❌ {package} no está instalado")
            missing_packages.append(version_spec)
    
    return missing_packages

# Verificar dependencias
print("🔍 Verificando dependencias...")
missing = check_dependencies()

if missing:
    print(f"\n📦 Faltan {len(missing)} dependencias:")
    for package in missing:
        print(f"   - {package}")
    
    install_choice = input("\n¿Desea instalar las dependencias faltantes? (s/n): ").lower().strip()
    
    if install_choice in ['s', 'si', 'y', 'yes']:
        print("\n📥 Instalando dependencias...")
        for package_spec in missing:
            package_name = package_spec.split('>=')[0]
            print(f"   Instalando {package_name}...")
            if install_package(package_spec):
                print(f"   ✅ {package_name} instalado correctamente")
            else:
                print(f"   ❌ Error instalando {package_name}")
        
        print("\n🔄 Reinicie el kernel del notebook tras la instalación.")
    else:
        print("\n⚠️ Algunas funcionalidades pueden no estar disponibles sin las dependencias.")
else:
    print("\n✅ Todas las dependencias están instaladas correctamente")

🔍 Verificando dependencias...
✅ dash está instalado
✅ plotly está instalado
✅ pandas está instalado
✅ numpy está instalado

✅ Todas las dependencias están instaladas correctamente


In [4]:
# Ejecutar la aplicación principal de análisis de modelos
print("🚀 Iniciando aplicación de análisis de modelos...")

try:
    # Importar y ejecutar la función principal
    from Analisis_modelos.main_visualizacion_modelos import main_visualizacion_modelos
    
    # Configuración de la aplicación
    config = {
        'json_path': None,  # Usar ruta automática
        'use_dash': True,   # Usar interfaz Dash
        'port': 8050,       # Puerto por defecto
        'debug': False      # Modo producción
    }
    
    print("⚙️ Configuración:")
    for key, value in config.items():
        print(f"   {key}: {value}")
    
    print("\n" + "="*60)
    print("🌐 APLICACIÓN WEB INTERACTIVA")
    print("="*60)
    print(f"📍 URL: http://localhost:{config['port']}")
    print("🔧 Controles disponibles:")
    print("   • Filtro por aeronave")
    print("   • Filtro por parámetro") 
    print("   • Selección de tipos de modelo")
    print("   • Filtros de predictores")
    print("   • Panel de información lateral")
    print("   • Tablas de métricas comparativas")
    print("\n⚡ Presione Ctrl+C en la terminal para detener")
    print("📖 Use los filtros para explorar los modelos de imputación")
    print("="*60)
    
    # Ejecutar aplicación
    main_visualizacion_modelos(**config)
    
except ImportError as e:
    print(f"❌ Error de importación: {e}")
    print("💡 Soluciones posibles:")
    print("   1. Instale las dependencias: pip install dash plotly pandas numpy")
    print("   2. Verifique que los módulos estén en la ruta correcta")
    print("   3. Reinicie el kernel del notebook")
    
    # Ofrecer versión alternativa
    print("\n🔄 Ejecutando versión simplificada por consola...")
    try:
        main_visualizacion_modelos(use_dash=False)
    except Exception as console_error:
        print(f"❌ Error en versión de consola: {console_error}")

except KeyboardInterrupt:
    print("\n\n⏹️ Aplicación detenida por el usuario")
    
except Exception as e:
    print(f"❌ Error ejecutando aplicación: {e}")
    print("💡 Verifique que:")
    print("   • El archivo JSON existe en la ruta correcta")
    print("   • Los datos tienen el formato esperado")
    print("   • No hay conflictos de puertos")

INFO:Analisis_modelos.main_visualizacion_modelos:Cargando datos desde: c:\Users\delpi\OneDrive\Tesis\ADRpy-VTOL\ADRpy\analisis\Results\modelos_completos_por_celda.json
INFO:Analisis_modelos.data_loader:Cargados 6 celdas con modelos
INFO:Analisis_modelos.data_loader:Cargados 6 celdas con detalles
INFO:Analisis_modelos.main_visualizacion_modelos:Datos cargados exitosamente
INFO:Analisis_modelos.data_loader:Cargados 6 celdas con modelos
INFO:Analisis_modelos.data_loader:Cargados 6 celdas con detalles
INFO:Analisis_modelos.main_visualizacion_modelos:Datos cargados exitosamente


🚀 Iniciando aplicación de análisis de modelos...
⚙️ Configuración:
   json_path: None
   use_dash: True
   port: 8050
   debug: False

🌐 APLICACIÓN WEB INTERACTIVA
📍 URL: http://localhost:8050
🔧 Controles disponibles:
   • Filtro por aeronave
   • Filtro por parámetro
   • Selección de tipos de modelo
   • Filtros de predictores
   • Panel de información lateral
   • Tablas de métricas comparativas

⚡ Presione Ctrl+C en la terminal para detener
📖 Use los filtros para explorar los modelos de imputación
Iniciando aplicación Dash en http://localhost:8050
Presione Ctrl+C para detener la aplicación


In [5]:
# Opción alternativa: Exploración manual de datos (para debugging)
print("\n" + "="*60)
print("🔧 MODO EXPLORACIÓN MANUAL")
print("="*60)

try:
    # Cargar datos manualmente para exploración
    if 'modelos_por_celda' in locals() and 'unique_values' in locals():
        print("✅ Datos ya cargados en memoria")
    else:
        print("📊 Cargando datos para exploración...")
        from Analisis_modelos.data_loader import load_models_data, extract_unique_values
        
        json_path = os.path.join('Results', 'modelos_completos_por_celda.json')
        modelos_por_celda, detalles_por_celda = load_models_data(json_path)
        unique_values = extract_unique_values(modelos_por_celda)
    
    # Estadísticas generales
    print(f"\n📈 ESTADÍSTICAS GENERALES:")
    print(f"   • Total de celdas con modelos: {len(modelos_por_celda)}")
    print(f"   • Aeronaves únicas: {len(unique_values['aeronaves'])}")
    print(f"   • Parámetros únicos: {len(unique_values['parametros'])}")
    print(f"   • Tipos de modelo: {len(unique_values['tipos_modelo'])}")
    
    # Análisis por tipo de modelo
    conteo_tipos = {}
    conteo_predictores = {}
    total_modelos = 0
    
    for celda, modelos in modelos_por_celda.items():
        for modelo in modelos:
            if isinstance(modelo, dict):
                total_modelos += 1
                tipo = modelo.get('tipo', 'desconocido')
                n_pred = modelo.get('n_predictores', 0)
                
                conteo_tipos[tipo] = conteo_tipos.get(tipo, 0) + 1
                conteo_predictores[n_pred] = conteo_predictores.get(n_pred, 0) + 1
    
    print(f"   • Total de modelos: {total_modelos}")
    
    print(f"\n📊 DISTRIBUCIÓN POR TIPO:")
    for tipo, cantidad in sorted(conteo_tipos.items()):
        porcentaje = (cantidad / total_modelos) * 100
        print(f"   • {tipo}: {cantidad} modelos ({porcentaje:.1f}%)")
    
    print(f"\n🔢 DISTRIBUCIÓN POR N° PREDICTORES:")
    for n_pred, cantidad in sorted(conteo_predictores.items()):
        porcentaje = (cantidad / total_modelos) * 100
        print(f"   • {n_pred} predictor{'es' if n_pred != 1 else ''}: {cantidad} modelos ({porcentaje:.1f}%)")
    
    # Ejemplo de modelo detallado
    print(f"\n🔍 EJEMPLO DE MODELO DETALLADO:")
    for celda_key, modelos in list(modelos_por_celda.items())[:1]:
        print(f"   Celda: {celda_key}")
        for i, modelo in enumerate(modelos[:1]):
            if isinstance(modelo, dict):
                print(f"   Modelo {i+1}:")
                print(f"     - Tipo: {modelo.get('tipo', 'N/A')}")
                print(f"     - Predictores: {modelo.get('predictores', [])}")
                print(f"     - Ecuación: {modelo.get('ecuacion_string', 'N/A')}")
                print(f"     - MAPE: {modelo.get('mape', 0):.3f}%")
                print(f"     - R²: {modelo.get('r2', 0):.3f}")
                print(f"     - Confianza: {modelo.get('Confianza', 0):.3f}")
                
                advertencia = modelo.get('Advertencia')
                if advertencia:
                    print(f"     - Advertencia: {advertencia}")
    
    print(f"\n💡 CONSEJOS DE USO:")
    print(f"   • Use la aplicación web para análisis interactivo completo")
    print(f"   • Filtre por aeronave primero, luego por parámetro")
    print(f"   • Compare modelos del mismo tipo para evaluación justa")
    print(f"   • Observe las advertencias para identificar limitaciones")
    print(f"   • Use hover para ver detalles completos de cada modelo")
    
except Exception as e:
    print(f"❌ Error en exploración manual: {e}")
    print("💡 Verifique que los datos estén cargados correctamente")


🔧 MODO EXPLORACIÓN MANUAL
✅ Datos ya cargados en memoria

📈 ESTADÍSTICAS GENERALES:
   • Total de celdas con modelos: 6
   • Aeronaves únicas: 3
   • Parámetros únicos: 5
   • Tipos de modelo: 7
   • Total de modelos: 174

📊 DISTRIBUCIÓN POR TIPO:
   • exp-1: 30 modelos (17.2%)
   • linear-1: 24 modelos (13.8%)
   • linear-2: 42 modelos (24.1%)
   • log-1: 30 modelos (17.2%)
   • poly-1: 12 modelos (6.9%)
   • poly-2: 6 modelos (3.4%)
   • pot-1: 30 modelos (17.2%)

🔢 DISTRIBUCIÓN POR N° PREDICTORES:
   • 1 predictor: 126 modelos (72.4%)
   • 2 predictores: 48 modelos (27.6%)

🔍 EJEMPLO DE MODELO DETALLADO:
   Celda: A7|payload
   Modelo 1:
     - Tipo: linear-1
     - Predictores: ['Potencia HP']
     - Ecuación: y = -4.285714285714278 + 1.4285714285714284*x0
     - MAPE: 3.626%
     - R²: 0.991
     - Confianza: 0.387

💡 CONSEJOS DE USO:
   • Use la aplicación web para análisis interactivo completo
   • Filtre por aeronave primero, luego por parámetro
   • Compare modelos del mismo 

In [ ]:
# Prueba específica para verificar las correcciones de visualización
print("\n" + "="*60)
print("🔧 PRUEBA DE CORRECCIONES DE VISUALIZACIÓN")
print("="*60)

try:
    # Recargar módulos para aplicar cambios
    import importlib
    
    if 'Analisis_modelos.plot_utils' in sys.modules:
        importlib.reload(sys.modules['Analisis_modelos.plot_utils'])
    if 'Analisis_modelos.data_loader' in sys.modules:
        importlib.reload(sys.modules['Analisis_modelos.data_loader'])
    
    from Analisis_modelos.plot_utils import create_interactive_plot
    from Analisis_modelos.data_loader import filter_models, prepare_plot_data
    
    if 'modelos_por_celda' in locals():
        # Probar con la primera aeronave que tenga datos
        if unique_values['aeronaves']:
            test_aeronave = unique_values['aeronaves'][0]  # A7
            
            # Obtener el primer parámetro disponible
            parametros_test = []
            for key in modelos_por_celda.keys():
                if key.startswith(f"{test_aeronave}|"):
                    param = key.split("|", 1)[1]
                    parametros_test.append(param)
            
            if parametros_test:
                test_parametro = parametros_test[0]  # payload
                print(f"🧪 Probando visualización: {test_aeronave} - {test_parametro}")
                
                # Filtrar modelos para esta combinación
                modelos_filtrados = filter_models(
                    modelos_por_celda,
                    aeronave=test_aeronave,
                    parametro=test_parametro
                )
                
                if modelos_filtrados:
                    celda_key = f"{test_aeronave}|{test_parametro}"
                    modelos = modelos_filtrados.get(celda_key, [])
                    
                    print(f"   📊 {len(modelos)} modelos encontrados")
                    
                    # Verificar datos de cada modelo
                    for i, modelo in enumerate(modelos[:2]):  # Solo primeros 2
                        if isinstance(modelo, dict):
                            print(f"   Modelo {i+1}: {modelo.get('tipo', 'N/A')}")
                            print(f"     - Predictores: {modelo.get('predictores', [])}")
                            print(f"     - Ecuación: {modelo.get('ecuacion_string', 'N/A')}")
                            
                            # Probar preparación de datos
                            df_orig, df_filt = prepare_plot_data(modelo)
                            if df_orig is not None:
                                print(f"     - Datos originales: {df_orig.shape}")
                                predictores_modelo = modelo.get('predictores', [])
                                if predictores_modelo:
                                    predictor = predictores_modelo[0]
                                    if predictor in df_orig.columns and test_parametro in df_orig.columns:
                                        # Verificar datos válidos
                                        df_clean = df_orig[[predictor, test_parametro]].replace('NaN', np.nan).dropna()
                                        print(f"     - Datos válidos: {len(df_clean)} puntos")
                                        if len(df_clean) > 0:
                                            print(f"     - Rango {predictor}: {df_clean[predictor].min():.2f} - {df_clean[predictor].max():.2f}")
                                            print(f"     - Rango {test_parametro}: {df_clean[test_parametro].min():.2f} - {df_clean[test_parametro].max():.2f}")
                                    else:
                                        print(f"     ⚠️ Columnas faltantes: {predictor} o {test_parametro}")
                            else:
                                print(f"     ❌ No se pudieron cargar los datos")
                    
                    # Intentar crear la visualización
                    print(f"\n🎨 Intentando crear visualización...")
                    try:
                        fig = create_interactive_plot(
                            modelos_filtrados,
                            test_aeronave,
                            test_parametro,
                            show_training_points=True,
                            show_model_curves=True
                        )
                        
                        # Verificar que se añadieron trazas
                        num_traces = len(fig.data)
                        print(f"   ✅ Visualización creada con {num_traces} trazas")
                        
                        if num_traces > 0:
                            trace_types = [trace.name for trace in fig.data]
                            print(f"   📈 Trazas: {', '.join(trace_types[:5])}{'...' if len(trace_types) > 5 else ''}")
                        else:
                            print(f"   ⚠️ No se añadieron trazas al gráfico")
                            
                    except Exception as plot_error:
                        print(f"   ❌ Error en visualización: {plot_error}")
                        import traceback
                        print(f"   Detalles: {traceback.format_exc()}")
                
                else:
                    print(f"   ❌ No se encontraron modelos filtrados")
            else:
                print(f"   ❌ No se encontraron parámetros para {test_aeronave}")
        else:
            print("   ❌ No hay aeronaves disponibles")
    else:
        print("⚠️ Datos no cargados. Ejecute las celdas anteriores primero.")
        
except Exception as e:
    print(f"❌ Error en prueba: {e}")
    import traceback
    print(f"Detalles: {traceback.format_exc()}")

## ✅ Correcciones Aplicadas

### Problemas Solucionados:

**🔧 Manejo de valores NaN:**
- Corrección en la carga del JSON para manejar valores `NaN` como strings
- Conversión automática de strings `'NaN'` a `np.nan` en DataFrames
- Filtrado de datos válidos antes de la visualización

**📊 Mejoras en visualización:**
- Validación robusta de existencia de columnas antes de graficar
- Limpieza de datos con `dropna()` para evitar errores de plotting
- Verificación de rangos válidos para las predicciones de modelos
- Manejo mejorado de coeficientes inválidos o faltantes

**📈 Robustez en curvas de modelos:**
- Validación de coeficientes antes de calcular predicciones
- Límites seguros para modelos exponenciales (evitar overflow)
- Mejor manejo de tipos de modelo desconocidos
- Logging detallado para debugging

### Para Probar las Correcciones:

1. **Ejecute la celda anterior** para verificar que los datos se cargan correctamente
2. **Ejecute la celda de prueba** para validar que la visualización funciona
3. **Lance la aplicación Dash** usando la celda principal de ejecución

Las ecuaciones deberían mostrarse correctamente ahora en las gráficas.

## Conclusiones y Próximos Pasos

### ✅ Funcionalidades Implementadas

**Módulo completo de análisis:**
- 🏗️ **Arquitectura modular** con separación clara de responsabilidades
- 📊 **Carga robusta** de datos JSON con manejo de errores
- 🔍 **Filtros encadenados** inteligentes para exploración progresiva
- 📈 **Visualizaciones interactivas** con hover detallado y métricas
- 🎛️ **Panel lateral** con información ampliada de modelos
- 🛡️ **Gestión de errores** con fallbacks y alternativas

**Capacidades de análisis:**
- ✅ Comparación visual de modelos sobre datos originales
- ✅ Resaltado de puntos de entrenamiento vs. datos completos
- ✅ Curvas de regresión calculadas para cada tipo de modelo
- ✅ Métricas comprensivas (MAPE, R², Correlación, Confianza)
- ✅ Identificación de modelos seleccionados para imputación
- ✅ Advertencias y validaciones de calidad de modelos

### 🚀 Próximos Pasos Recomendados

**Extensiones técnicas:**
1. **Soporte para modelos de 2+ predictores** con visualizaciones 3D
2. **Exportación de resultados** a PDF/Excel con reportes automáticos
3. **Comparación estadística** entre métodos de imputación
4. **Integración con pipeline** para análisis en tiempo real
5. **Caching inteligente** para mejorar rendimiento con datasets grandes

**Mejoras de interfaz:**
1. **Temas visuales** personalizables y modo oscuro
2. **Tooltips explicativos** para métricas técnicas
3. **Atajos de teclado** para navegación rápida
4. **Favoritos** para guardar combinaciones de filtros frecuentes
5. **Historial** de sesiones de análisis

**Validaciones avanzadas:**
1. **Detección de overfitting** mediante validación cruzada
2. **Análisis de residuos** para evaluar supuestos del modelo
3. **Pruebas de normalidad** y homogeneidad de varianzas
4. **Intervalos de confianza** para predicciones
5. **Métricas de robustez** ante outliers

### 💡 Recomendaciones de Uso

**Para análisis exploratorio:**
- Comience filtrando por aeronave de interés
- Compare tipos de modelos similares (mismo n° de predictores)
- Observe las advertencias para identificar limitaciones
- Use el panel lateral para entender ecuaciones y métricas

**Para validación de modelos:**
- Priorice la interpretabilidad física sobre métricas numéricas
- Verifique que los predictores tengan sentido causal
- Compare múltiples métricas, no solo R² o MAPE
- Considere el tamaño de muestra de entrenamiento

**Para toma de decisiones:**
- Evalúe el balance entre precisión y simplicidad
- Considere la confianza ajustada por complejidad
- Verifique la robustez ante diferentes familias de aeronaves
- Documente las razones de selección de modelos

¡El módulo está listo para usar y puede extenderse según las necesidades específicas del proyecto!